# Censo de poblacion y vivienda 2020

## Librerias y configuraciones

In [14]:
import os
import pandas as pd
import random as rnd
import numpy as np
from math import ceil
import regex as re

In [2]:
# Variable definida para imprimir información adicional en la libreta.
verbose = True

## Importar archivos

In [3]:
# DATA_DIR es una variable de entorno en mi computadora a la cual se le 
# asignó el valor de el dicrectorio donde se guardan los datos a procesar.
data_dir = os.getenv('DATA_DIR')

# La información censal la guardo en una carpeta llamada EventosCensales 
# dentro del directorio de datos.
censales_dir = 'EventosCensales'

# Se respetó la estructura de carpetas original que se obtiene al descomprimir 
# el archivo zip que se obtiene de INEGI. Algunas veces se modifican estos 
# archivos asi que pueden cambiar. En este caso el zip se descomprimió en una 
# carpeta llamada iter_00_cpv2020.
censo_2020_dir = 'iter_00_cpv2020'

# Los datos se encuentran dentro de la carpeta conjunto_de_datos y el descriptor 
# de cada uno de los campos se encuentra en diccionario_datos.
subdir_datos = 'conjunto_de_datos'
subdir_descriptor = 'diccionario_datos'

# El nombre del archivo donde se encentran los datos es 
# 'conjunto_de_datos_iter_00CSV20.csv' y descriptor de la base tiene el nombre 
# 'diccionario_datos_iter_00CSV20.csv'. Los dos vienen en formato csv.
archivo_datos = 'conjunto_de_datos_iter_00CSV20.csv'
archivo_descriptor = 'diccionario_datos_iter_00CSV20.csv'

# Concatenar las rutas de archivos
ruta_de_archivo = os.path.join(data_dir, 
                               censales_dir, 
                               censo_2020_dir, 
                               subdir_datos, 
                               archivo_datos)

ruta_de_descriptor = os.path.join(data_dir, 
                                  censales_dir, 
                                  censo_2020_dir, 
                                  subdir_descriptor, 
                                  archivo_descriptor)

if verbose:

    print(f'El archivo de datos existe: {os.path.isfile(ruta_de_archivo)}')
    print(f'El descriptor de base de datos existe: {os.path.isfile(ruta_de_descriptor)}')


El archivo de datos existe: True
El descriptor de base de datos existe: True


In [4]:
# Importar archivo de datos
raw_censo_2020 = pd.read_csv(ruta_de_archivo)

C:\Users\Lorrain\AppData\Local\Temp\ipykernel_5440\2739558870.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_censo_2020 = pd.read_csv(ruta_de_archivo)


In [5]:
# Importar descriptor de datos
nuevos_nombres = {
    'Núm.':'NUM', 
    'Indicador':'INDICADOR',
    'Descripción':'DESCRIPCION',
    'Mnemónico':'NOMCOL',
    'Rangos':'RANGOS',
    'Longitud':'LONGITUD'
}

diccionario = pd.read_csv(ruta_de_descriptor, skiprows=4, usecols=list(nuevos_nombres.keys()))
diccionario.rename(columns=nuevos_nombres, inplace=True)

## Análisis inicial

La mayor parte de las columnas se importa como objeto, lo que significa que existen diferentes tipos de datos dentro de cada una de las columnas. Para homogeneizar los datos se tiene que hacer un analisis mas detallado para definir que tipo de dato debe de ser cada una de las columnas y cuales son los valores que no son de este tipo en cada una de ellas.

In [6]:
# Primera descripcion
raw_censo_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195662 entries, 0 to 195661
Columns: 286 entries, ENTIDAD to TAMLOC
dtypes: int64(6), object(280)
memory usage: 426.9+ MB


El primer registro es un buen acercamiento al tipo de dato que contiene cada una de las columnas. Representa el total nacional, y como tal hay columnas que no aplican. Tal es el caso de las columnas que definen el nombre y clave del Estado, Municipio, y Localidad. No contienen tampoco longitud y latiud, estos datos solo estan presentes en los registros que representan localidades especificas.

In [7]:
# Diccionario cuyas llaves son el nombre de columna y valores del primer registro 
# del dataframe.
ejemplo_datos = {key : value for key, value in 
                zip(raw_censo_2020.columns, raw_censo_2020.iloc[0])}

if verbose:
    print(f'Numero de columnas {len(ejemplo_datos.items())}')

Numero de columnas 286


In [8]:
# En general las libretas jupyter no imprimen un texto de mas de ciertas lineas.
# para ver todos los registros hay que cambiar los valores dentro de los
# corchetes ([0:20]) de la linea comentada para recorrerlos todos. Hay 286 pares 
# llave: valor en el diccionario.

#for item in list(ejemplo_datos.items())[0:20]: print(item)  

Existen tres tipos de columnas que deben estar en formato string. El primer tipo es el que contiene los nombres de los Estados, Municipios y Localidades. El segundo tipo contiene las claves de los diferentes niveles de entidad. Y el tercero contiene las coodenadas geograficas en formato de grados minutos y seguntos. Las columnas que deberian ser texto son las siguientes:

['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD', 'LATITUD']

El resto de las columnas contiene valores numericos. Casi todas tienen el formato de numeros enteros pero 6 que contienen promedios y una que contiene la relacion que existe entre hombres y mujeres de la delimitación geoestadistica. Estos campos contienen decimales por lo que deben de tener el formato Float. Las columnas con valores decimales son:

['REL_H_M', 'PROM_HNV', 'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', 'PROM_OCUP', 'PRO_OCUP_C']

El resto de las columnas contienen numeros enteros.

In [9]:
# Todas las columnas
columnas_censo_2020 = list(raw_censo_2020.columns)

# Columnas de texto
columnas_string = ['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 
                   'LONGITUD', 'LATITUD']
# Columnas con valores decimales
columnas_decimal = ['REL_H_M', 'PROM_HNV', 'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', 
                    'PROM_OCUP', 'PRO_OCUP_C']

# El resto de las columnas son numeros enteros
columnas_entero = [i for i in columnas_censo_2020 
                   if i not in columnas_string + columnas_decimal]

Existen dos valores no numericos en las columnas con formato de numeros enteros y tambien de decimales. Estos valores hacen que el programa represente como objeto todos los registros. Los valores de texto que hay que sustituir son los siguientes:

['*', 'N/D']

La columna ALTITUD tiene 32 valores no numericos. En teoria deberia contener valores enteros y va a ser tratado como un caso aparte.

In [10]:
# Tarda un poco en calcularse esta celda
valores_string_por_columna = {}
for column in columnas_censo_2020:
        # Se quitaron las columnas con nombres pero se dejaron las columnas con claves
        # alfanumericas
        if column not in ['NOM_ENT', 'NOM_MUN', 'NOM_LOC', 'LONGITUD', 'LATITUD']:
                valores_no_numericos = tuple(raw_censo_2020[~raw_censo_2020[column]
                                                            .astype('string')
                                                            .str.strip()
                                                            .str.replace('.', '', regex = False)
                                                            .str.isnumeric()]
                                                            [column].unique())
                if valores_no_numericos in valores_string_por_columna.keys():
                        valores_string_por_columna[valores_no_numericos] += [column]
                
                else:
                        valores_string_por_columna[valores_no_numericos] = [column]


In [11]:
# Registros con valores de texto
#raw_censo_2020[raw_censo_2020.eq('*').any(1)]
#raw_censo_2020[raw_censo_2020.eq('N/D').any(1)]


In [12]:
# Resumen valores de texto

if verbose:
    ND_verdaderos = raw_censo_2020.eq('N/D').any(axis = 1).value_counts()[1]
    asterisco_verdaderos = raw_censo_2020.eq('*').any(axis = 1).value_counts()[1]
    
    elementos_por_linea = 7
    for llave, valor in valores_string_por_columna.items():
        iteraciones_llaves = len(llave) / elementos_por_linea
        iteraciones_valores = len(valor) / elementos_por_linea

        if iteraciones_llaves < 1: iteraciones_llaves  = 1
        else: iteraciones_llaves = ceil(iteraciones_llaves)

        if iteraciones_valores < 1: iteraciones_valores = 1
        else: iteraciones_valores = ceil(iteraciones_valores)

        
        inicial, final = 0, elementos_por_linea
        
        print('Columnas:')
        for columnas_similares in range(0, iteraciones_valores):

            print(valor[inicial: final])
            inicial += elementos_por_linea
            final += elementos_por_linea

        print('Valores de texto:')   
        inicial, final = 0, elementos_por_linea
        for valores_llave in range(0, iteraciones_llaves):

            print(llave[inicial:final])
            inicial += elementos_por_linea
            final += elementos_por_linea
        print()
    
    print(f'Filas en los que aparece el valor N/D: {ND_verdaderos}')
    print(f'Filas en los que aparece el valor *: {asterisco_verdaderos}')

Columnas:
['ENTIDAD', 'MUN', 'LOC', 'POBTOT', 'VIVTOT', 'TVIVHAB']
Valores de texto:
()

Columnas:
['ALTITUD']
Valores de texto:
('-006', '-001', '-002', '-005', '00-2', '-012', '-007')
('-008', '-009', '-003', '-010', '00-1', '00-4', -8.0)
(-2.0, -6.0, -5.0, -1.0, -7.0, '-004', '00-3')
('-013', -3.0, -18.0, -4.0, -9.0, '00-8', '00-5')
(-10.0, -11.0, -12.0, -15.0)

Columnas:
['POBFEM', 'POBMAS', 'TAMLOC']
Valores de texto:
('*',)

Columnas:
['P_0A2', 'P_0A2_F', 'P_0A2_M', 'P_3YMAS', 'P_3YMAS_F', 'P_3YMAS_M', 'P_5YMAS']
['P_5YMAS_F', 'P_5YMAS_M', 'P_12YMAS', 'P_12YMAS_F', 'P_12YMAS_M', 'P_15YMAS', 'P_15YMAS_F']
['P_15YMAS_M', 'P_18YMAS', 'P_18YMAS_F', 'P_18YMAS_M', 'P_3A5', 'P_3A5_F', 'P_3A5_M']
['P_6A11', 'P_6A11_F', 'P_6A11_M', 'P_8A14', 'P_8A14_F', 'P_8A14_M', 'P_12A14']
['P_12A14_F', 'P_12A14_M', 'P_15A17', 'P_15A17_F', 'P_15A17_M', 'P_18A24', 'P_18A24_F']
['P_18A24_M', 'P_15A49_F', 'P_60YMAS', 'P_60YMAS_F', 'P_60YMAS_M', 'REL_H_M', 'POB0_14']
['POB15_64', 'POB65_MAS', 'P_0A4', 'P_0

Las columnas LONGITUD y LATITUD tiene la informacion de las coordenadas en formato de grados minutos y segundos. Existen registros sin un valor en estas columnas pero son registros que representan la informacion total nacional, de los estados, municipios y la informacion de localidades de una y dos viviendas.

Los registros que si contienen información en estas columnas contienen el mismo formato de texto. Una vez que se les quitan los espacios en blanco antes y despues de los textos (.strip), contienen dos o tres digitos seguidos del simbolo de grado (°), seguido de otros dos digitos y el simbolo de minutos ('), seguido de tres digitos mas, un punto (.), tres digitos mas y el simbolo de segundos ("), seguido de un espacio en blanco y la letra W en la longitud y N para la latitud.

In [89]:
#raw_censo_2020[raw_censo_2020.LATITUD.isna()].NOM_LOC.unique()
#raw_censo_2020[raw_censo_2020.LONGITUD.isna()].NOM_LOC.unique()

In [88]:
# Comprobar formato de longitud
raw_censo_2020.LATITUD\
    .str.strip()\
    .str.contains(r'^\d{2}°\d{2}\'\d{2}\.\d{3}\"\sN$')\
    .value_counts(dropna = False)

True    189432
NaN       6230
Name: LATITUD, dtype: int64

In [73]:
#Comprobar formato de latitud
raw_censo_2020.LONGITUD\
    .str.strip()\
    .str.contains(r'^\d{2,3}°\d{2}\'\d{2}\.\d{3}\"\sW$')\
    .value_counts(dropna = False)

True    189432
NaN       6230
Name: LONGITUD, dtype: int64

In [79]:
#diccionario.columns                   # Columnas del diccionario
#list(diccionario.NOMCOL.unique())[:]  # Columnas del iter

In [90]:
# Información de columnas del Censo 2020
columna_de_interes = 'ENTIDAD'  # Poner columna de interes aqui.
info_columna = diccionario[diccionario.NOMCOL.isin([columna_de_interes])]

for columna in info_columna.columns:
    print(f'{columna}:')
    print(info_columna[columna].values)


NUM:
[1]
INDICADOR:
['Clave de entidad federativa']
DESCRIPCION:
['Código que identifica a la entidad federativa. El código 00 identifica a los registros con los totales a nivel nacional.']
NOMCOL:
['ENTIDAD']
RANGOS:
['00…32']
LONGITUD:
[2]


## Cambiar el formato a columnas

### Columnas con claves

Las columnas con las claves alfanumericas de los estados, municipios y localidades fueron importadas en su mayoria como numeros. Se van a cambiar a formato de numero y se les va a reformatear. para que tengan el numero correcto de caracterers necesario para poder concatenarlas.

Entidad es una clave que debe de tener 2 posiciones aunque su valor numerico sea 1. En este caso la clave correcta que debe de tener es '01'. La clave de los municipios esta conformada por 3 posiciones, y la de localidad por 4. Al igual que la clave del estado, si el municipio o la localidad tuvieran asignado el numero 1 para representarlos, las claves correctas deben de ser '001' y '0001' segun sea el caso.

In [54]:
# Reformatear columnas con claves alfa numericas.
for string_column, keylength in zip(['ENTIDAD', 'MUN', 'LOC'], [2, 3, 4]):
    raw_censo_2020[string_column] = raw_censo_2020[string_column].astype('string').str.zfill(keylength)



In [67]:
# Resultado
if verbose:
    print('Claves de entidad, municipio y localidad corregidas:', end='\n\n')
    print(raw_censo_2020.iloc[rnd.randint(0,raw_censo_2020.shape[0]),0:5])

Claves de entidad, municipio y localidad corregidas:

ENTIDAD                     16
NOM_ENT    Michoacán de Ocampo
MUN                        010
NOM_MUN                Arteaga
LOC                       0881
Name: 96987, dtype: object


### Columnas de coordenadas

Las columnas LONGITUD Y LATITUD se conservarán tal y como se encuentran pero se procesaran de dos maneras para que existan columnas en el formato que los Sistemas de Información Geografica piden para geolocalizar puntos en el mapa. 

Los formatos que se aceptan son columnas numericas donde se definen de manera separada los grados, los minutos y los segundos para la longitud y la latiud. Y una sola columna en grados decimales, donde el numero entero representa los grados, y los digitos decimales representan la porcion de minutos y segundos restantes.

Antes, los programas GIS necestaban las coordenadas en grados decimales para poder renderizar el mapa. Hoy en dia tambien aceptan la información de grados, minutos y segundos en columnas aparte y lo calculan ellos de manera automatica. Se generarán las columnas para representar las localidades en los dos formatos.


In [ ]:
# Grados, minutos, y segundos en columnas separadas.

In [ ]:
# Grados decimales

### Columnas numericas

### Columna de altitud

## Pruebas...

In [9]:

column_dtype = {}
for column in raw_censo_2020.columns:

    if column in columnas_string:
        column_dtype[column] = 'string'

    elif column in columnas_decimal:
        column_dtype[column] = 'Float64'

    else:
        column_dtype[column] = 'Int64'


In [18]:

for string_column in columnas_string:
    if string_column not in ['ENTIDAD', 'MUN', 'LOC']:
        raw_censo_2020[string_column] = raw_censo_2020[string_column].astype("string")

for numeric_column in raw_censo_2020.columns:

    if numeric_column not in columnas_string + ['ALTITUD']:
        
        raw_censo_2020[numeric_column].replace(to_replace = ['*', 'N/D'], value = np.nan, inplace = True)

        if numeric_column in columnas_decimal:
            raw_censo_2020[numeric_column] = raw_censo_2020[numeric_column].astype('Float64')
        
        else:
            raw_censo_2020[numeric_column] = raw_censo_2020[numeric_column].astype('Int64')





In [19]:
columnas_string + columnas_decimal

['ENTIDAD',
 'NOM_ENT',
 'MUN',
 'NOM_MUN',
 'LOC',
 'NOM_LOC',
 'LONGITUD',
 'LATITUD',
 'REL_H_M',
 'PROM_HNV',
 'GRAPROES',
 'GRAPROES_F',
 'GRAPROES_M',
 'PROM_OCUP',
 'PRO_OCUP_C']

In [20]:
#raw_censo_2020.iloc[:,0:8].info()
#raw_censo_2020.loc[:,columnas_decimal].info()
raw_censo_2020.loc[:,raw_censo_2020.columns.drop(columnas_decimal+columnas_string)].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195662 entries, 0 to 195661
Columns: 271 entries, ALTITUD to TAMLOC
dtypes: Int64(270), object(1)
memory usage: 454.9+ MB


In [9]:
descriptor.RANGOS.unique()

array(['00…32', 'Alfanumérico', '000…570', '0000…9999', 'Caracter',
       '0...999999999', '0…999999999', '0.,.999999999', '0..999999999',
       '0...99999999', '0… 999999999', '01..14'], dtype=object)

In [13]:
nuevos_nombres = {
    'Núm.':'NUM', 
    'Indicador':'INDICADOR',
    'Descripción':'DESCRIPCION',
    'Mnemónico':'NOMCOL',
    'Rangos':'RANGOS',
    'Longitud':'LONGITUD'
}

descriptor = pd.read_csv(ruta_de_descriptor, skiprows=4, usecols=list(nuevos_nombres.keys()))
descriptor.rename(columns=nuevos_nombres, inplace=True)

In [43]:
raw_censo_2020.columns

Index(['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT',
       ...
       'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ',
       'VPH_SINRTV', 'VPH_SINLTC', 'VPH_SINCINT', 'VPH_SINTIC', 'TAMLOC'],
      dtype='object', length=286)